In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Изучение данных

Откроем файл с исходными данными и выведем первые пять строк на экран

In [3]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [4]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разделение данных на выборки

Сначала выделим признаки и целевой признак

In [5]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

Теперь разделим признаки и целевой признак на тренировочную, валидационную и тестовую выборки в соотношении 3:1:1

**features**

In [6]:
features_train, features_valid_test = train_test_split(features, test_size=0.4, random_state=2021)

In [7]:
features_valid, features_test = train_test_split(features_valid_test, test_size=0.5, random_state=2021)

**target**

In [8]:
target_train, target_valid_test = train_test_split(target, test_size=0.4, random_state=2021)

In [9]:
target_valid, target_test = train_test_split(target_valid_test, test_size=0.5, random_state=2021)

В итоге:

Для признаков(features):
- features_train - тренировочная выборка (размер 1928)
- features_valid - валидационная выборка (размер 643)
- features_test - тестовая выборка (размер 643)

Для целевого признака(target):
- target_train - тренировочная выборка (размер 1928)
- target_valid - валидационная выборка (размер 643)
- target_test - тестовая выборка (размер 643)


## Исследование моделей

Необходимо выбрать модель и гиперпараметры, которые обеспечивают лучшее выполнение (самое большое значение accuracy) на валидационном наборе

### DecisionTreeClassifier

In [19]:
best_result = 0
best_tree_model = None
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=2021, max_depth=depth)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predict)
    if accuracy > best_result:
        best_result = accuracy
        best_depth = depth
        best_tree_model = model
print(f'Лучший результат: {best_result}')
print(f'Глубина дерева: {best_depth}')

Лучший результат: 0.80248833592535
Глубина дерева: 10


**Вывод:** Лучший результат accuracy = 0.802 достигается при глубине дерева 10

### RandomForestClassifier

In [20]:
best_result = 0
best_estmators = 0
best_forest_model = None
for estimators in range(1, 20):
    for depth in range(1, 20):
        model = RandomForestClassifier(random_state=2021, n_estimators=estimators, max_depth=depth)
        model.fit(features_train, target_train)
        predict = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predict)
        if accuracy > best_result:
            best_result = accuracy
            best_estmators = estimators
            best_depth = depth
            best_forest_model = model
print(f'Лучший результат: {best_result}')
print(f'Количество деревьев: {best_estmators}')
print(f'Глубина: {best_depth}')

Лучший результат: 0.8087091757387247
Количество деревьев: 5
Глубина: 13


**Вывод:** Лучший результат accuracy = 0.808 достигается при количестве деревьев 5 и глубине 13


### LogisticRegression

In [21]:
model = LogisticRegression(random_state=2021)
model.fit(features_train, target_train)
predict = model.predict(features_valid)
accuracy_score(target_valid, predict)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7045101088646968

**Вывод:** accuracy = 0.704 


## Проверка на тестовой выборке

Проверим значение метрики accuracy на моделях, которые дали лучший результат на валидационной выборке

**DecisionTreeClassifier**

In [22]:
predict = best_tree_model.predict(features_test)
accuracy_score(target_test, predict)

0.7993779160186625

**RandomForestClassifier**

In [23]:
predict = best_forest_model.predict(features_test)
accuracy_score(target_test, predict)

0.7962674961119751

**LogisticRegression**

In [24]:
model = LogisticRegression(random_state=2021)
model.fit(features_train, target_train)
predict = model.predict(features_test)
accuracy_score(target_test, predict)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6936236391912908

## Общий вывод

Луший результат дала модель DecisionTreeClassifier с глубиной дерева 10